In [1]:
import pandas as pd
import numpy as np

import torch

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, MultiHorizonMetric
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
data_path = "./Refined_Data/Grouped_Data/Input_Data.csv"

group = [
    [4],[11, 12],[34],[40],[10],[42],[41]                                   # 0~ 6
    ,[6, 8, 13, 17, 26, 48, 53, 55, 56],[7, 18],[27, 57],[35, 46, 47],[25]  # 7 ~ 11
    ,[1, 31],[9, 32],[3],[33]                                               # 12 ~ 15
    ,[2, 14, 22, 37, 44, 52, 54],[15],[38, 58, 43],[29, 39],[45],[23]       # 16 ~ 21
    ,[5],[16, 24],[19, 20, 21, 49, 50, 51],[28, 36, 60],[59], [30]          # 22 ~ 26
]

In [3]:
data = pd.read_csv(data_path, parse_dates = ["date_time"])

data['num']     =   data['num'].apply(str)
data['Week']    =   data['Week'].apply(str)
data['24Hour']  =   data['24Hour'].apply(str)
data['holiday'] =   data['holiday'].apply(str)
data['Weekend'] =   data['Weekend'].apply(str)
data['energy_group'] = data['energy_group'].apply(str)
data['hour_cat']=   data['hour_cat'].apply(str)

In [4]:
data_bag = [data.loc[data["energy_group"] == str(i)].copy() for i in range(len(group))]

models = []

In [6]:
for idx in range(len(data_bag)):

    data = data_bag[idx]

    torch.cuda.empty_cache()

    max_prediction_length = 168
    max_encoder_length = 336
    training_cutoff = data["time_idx"].max() - max_prediction_length

    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= training_cutoff],
        time_idx="time_idx",
        target="kWH",
        group_ids=["num"],
        min_encoder_length=max_encoder_length,
        max_encoder_length=max_encoder_length,
        min_prediction_length=max_prediction_length,
        max_prediction_length=max_prediction_length,
        static_categoricals=["num", "energy_group"],
        static_reals=["non_electric_aircondition", "sunlight"],
        time_varying_known_categoricals=["Week", "24Hour", "holiday", "Weekend", "hour_cat"],
        time_varying_known_reals=["C", "m/s", "wet", "mm", "hr", "time_idx", "perceived_temperature", "discomfort_index"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=["kWH"],
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
    )

    validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

    batch_size = 64  # set this between 32 to 128
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=8)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=1)

    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=500,
        gpus=1,
        weights_summary="top",
        gradient_clip_val=0.1,
        limit_train_batches=10, 
        callbacks=[lr_logger, early_stop_callback],
        logger=logger,
    )


    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.01,
        hidden_size=64,
        lstm_layers = 1,
        attention_head_size=4,
        dropout=0.15,
        hidden_continuous_size=8,
        output_size=1,
        loss=SMAPE(),
        log_interval=0,
        reduce_on_plateau_patience=4,
    )

    # fit network
    trainer.fit(
        tft,
        train_dataloader=train_dataloader,
        val_dataloaders=val_dataloader,
    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | SMAPE                           | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 257   
3  | prescalers                         | ModuleDict                      | 240   
4  | static_variable_selection          | VariableSelectionNetwork        | 9.0 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 20.0 K
6  | decoder_variable_selection         | VariableSelectionNetwork        | 18.1 K
7  | static_context_variable_selection  | GatedResidualNetwork            | 16.8 K
8  | static_context_initial_hidden_lstm | GatedR